In [1]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
# from actor import ActorNetwork
# from rbfActor import RbfActorNetwork
# from critic import CriticNetwork
from actorCma import ActorCmaNetwork 
# from replay_buffer import ReplayBuffer
# from ounoise import OUNoise
import gym, time, cma
from Envs.reaching import ReachingEnv
# from Envs.throwing import ThrowingEnv
import matplotlib.pyplot as plt

In [2]:
MAX_EPISODE = 1000
MAX_TIME = 200
RANDOM_SEED = 1926

goal_pos = np.array([[-1.7691047 , -1.76426373],
       [-1.81476041,  1.25572033],
       [ 1.97538345,  1.7239961 ],
       [ 0.49885795,  1.82511657],
       [-1.45703216,  1.39941234],
       [ 0.49100693,  0.12822174],
       [ 0.14809867, -1.31716354],
       [-1.30413931,  1.79390377],
       [-0.60087002,  1.90940639],
       [ 1.29273111,  1.85736147]])

env = ReachingEnv(include_t = True)
# env = ThrowingEnv(include_t = True)
tf.set_random_seed(RANDOM_SEED)
state_dim = env.observation_space.shape[0]
action_dim = env.action_space.shape[0]
action_bound = env.action_space.high
sess = tf.Session()

hidden_layer_dim = [40,30]
if len(hidden_layer_dim) == 2:
    bound1 = state_dim * hidden_layer_dim[0]
    bound2 = bound1 + hidden_layer_dim[0]*hidden_layer_dim[1]
    bound3 = bound2 + hidden_layer_dim[1]*action_dim
    bound4 = bound3 + hidden_layer_dim[0]
    bound5 = bound4 + hidden_layer_dim[1]
    total_length = bound5
elif len(hidden_layer_dim) == 1:
    bound1 = state_dim * hidden_layer_dim[0]
    bound2 = bound1 + hidden_layer_dim[0]*action_dim
    bound3 = bound2 + hidden_layer_dim[0]
    total_length = bound3

In [3]:
actor = ActorCmaNetwork(sess, state_dim, action_dim, action_bound, hidden_layer_dim = hidden_layer_dim, seed = RANDOM_SEED)

In [4]:
def cost_fcn_cma(network_vec, env):
    state = env.reset()
    net_dict = wrap_network(network_vec, len(hidden_layer_dim))
    reward_list = []
    for t in range(MAX_TIME):
        action = actor.predict(np.reshape(state, (-1, state_dim)), net_dict)
        action = np.clip(action, -action_bound, action_bound)
        action = np.reshape(action, action_dim)
        next_state, reward, done, info = env.step(action)
        reward_list.append(reward)
        if done:
            break
        state = next_state
#         if t%100 == 0:
# #             print(action)
    return -np.sum(reward_list)

In [5]:
def wrap_network(network_vector, net_depth = 2):
    
    net_dict = {}
    if net_depth == 2:
        bound1 = state_dim*hidden_layer_dim[0]
        bound2 = bound1 + hidden_layer_dim[0]*hidden_layer_dim[1]
        bound3 = bound2 + hidden_layer_dim[1]*action_dim
        bound4 = bound3 + hidden_layer_dim[0]
        bound5 = bound4 + hidden_layer_dim[1]
        net_dict['weights1'] = np.reshape(network_vector[0:bound1], [state_dim, hidden_layer_dim[0]])
        net_dict['weights2'] = np.reshape(network_vector[bound1:bound2], [hidden_layer_dim[0], hidden_layer_dim[1]])
        net_dict['weights3'] = np.reshape(network_vector[bound2:bound3], [hidden_layer_dim[1], action_dim])
        net_dict['bias1'] = np.reshape(network_vector[bound3:bound4], [hidden_layer_dim[0]])
        net_dict['bias2'] = np.reshape(network_vector[bound4:bound5], [hidden_layer_dim[1]])
    elif net_depth == 1:
        bound1 = state_dim * hidden_layer_dim[0]
        bound2 = bound1 + hidden_layer_dim[0]*action_dim
        bound3 = bound2 + hidden_layer_dim[0]
        net_dict['weights1'] = np.reshape(network_vector[0:bound1], [state_dim, hidden_layer_dim[0]])
        net_dict['weights2'] = np.reshape(network_vector[bound1:bound2], [hidden_layer_dim[0], action_dim])
        net_dict['bias1'] = np.reshape(network_vector[bound2:bound3], [hidden_layer_dim[0]])
    return net_dict



In [6]:
init_policy_vec = np.random.normal(0., 1., total_length)
init_policy_vec[bound2:bound3] /= 333. 
# sigma0 = np.ones(total_length)
# sigma0[bound1:bound2] /= 33.
sigma0 = 1.
es = cma.CMAEvolutionStrategy(init_policy_vec,sigma0)
objfun = lambda network_vec: cost_fcn_cma(network_vec, env)
logger = cma.CMADataLogger().register(es)
es.optimize(objfun, verb_disp = True, iterations = 40)

/usr/local/lib/python2.7/dist-packages/cma.py:2938: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if x0 == str(x0):


(12_w,25)-aCMA-ES (mu_w=7.3,w_1=23%) in dimension 1530 (seed=935492, Tue Jun  6 13:15:35 2017)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     25 1.922868886973092e+02 1.0e+00 9.95e-01  1e+00  1e+00 0:15.1
    2     50 1.725685089320600e+02 1.0e+00 9.90e-01  1e+00  1e+00 0:29.4
    3     75 1.492094499221072e+02 1.0e+00 9.85e-01  1e+00  1e+00 0:44.4
    4    100 1.037170801195915e+02 1.0e+00 9.80e-01  1e+00  1e+00 0:59.1
    5    125 1.344995427315829e+02 1.0e+00 9.76e-01  1e+00  1e+00 1:13.1
    6    150 1.875021948930822e+02 1.0e+00 9.71e-01  1e+00  1e+00 1:26.9
    7    175 6.356764495640721e+01 1.0e+00 9.67e-01  1e+00  1e+00 1:40.3
    8    200 7.073946384127721e+01 1.0e+00 9.63e-01  1e+00  1e+00 1:56.9
    9    225 1.168150110803258e+02 1.0e+00 9.59e-01  1e+00  1e+00 2:12.7
   10    250 4.710009763745898e+01 1.0e+00 9.56e-01  1e+00  1e+00 2:28.3
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
   11    275 5.654974574254891e

(array([-1.99143779,  4.24092834, -9.9389795 , ...,  2.45039636,
         0.95360646, -0.08335201]),
 29.754867611634623,
 891,
 1000,
 40,
 array([-0.66365851,  4.65902905, -8.89400246, ...,  3.01462845,
         0.64587625, -0.71363907]),
 array([ 0.87681366,  0.87680533,  0.8768039 , ...,  0.87692496,
         0.87676647,  0.87679917]))